# Análise de uma Matriz de Insumo-Produto Brasileira

---

## Conceitos 

- **Matriz Insumo-Produto:** um modelo que representa os setores da economia de um país ou de uma região através de uma matriz, para prever o impacto de alterações de um setor sobre os outros e de consumidores, do governo e de fornecedores estrangeiros sobre a economia, desenvolvido por [Wassily Leontief](https://pt.wikipedia.org/wiki/Wassily_Leontief). Cada coluna da matriz representa o valor monetário de insumos (*inputs*) de um setor e cada linha representa o valor monetário das saídas (*outputs*) de um setor, mostrando as relações de dependência dos setores da economia. 


- **Matriz $A$:** Matriz que relaciona os valores de cada posição na matriz ($z_{ij}$) com a produção total do setor $i$. Matematicamente temos: 

$$x_i \equiv \sum_{j=1}^n z_{ij} + y_i ~~~~~\forall~i,j = 1, 2, ..., n$$

$$a_{ij} = \frac{z_{ij}}{x_i} ~~~~~~~~~~\forall~i,j = 1, 2, ..., n$$

Juntando as duas equações podemos reescrever da seguinte forma:

$$x_i = \sum_{j=1}^n a_{ij}x_j + y_i$$

Em termos matriciais temos:

$$x = Ax + y$$

$$\begin{bmatrix}
x_1\\
x_2\\
...\\
x_i\\
...\\
x_n
\end{bmatrix} =
\begin{bmatrix}
a_{11} & a_{12} & ... & a_{1i} & ... & a_{1n}\\
a_{21} & a_{22} & ... & a_{2i} & ... & a_{2n}\\
...    & ...    & ... & ...    & ... & ...\\
a_{i1} & a_{i2} & ... & a_{ii} & ... & a_{in}\\
...    & ...    & ... & ...    & ... & ...\\
a_{n1} & a_{n2} & ... & a_{ni} & ... & a_{nn}
\end{bmatrix}
\begin{bmatrix}
x_1\\
x_2\\
...\\
x_i\\
...\\
x_n
\end{bmatrix} +
\begin{bmatrix}
y_1\\
y_2\\
...\\
y_i\\
...\\
y_n
\end{bmatrix}$$

Assim podemos fazer manipulações algébricas:

$$x - Ax = y$$
$$(I - A)x = y$$
$$x = (I - A)^{-1}y$$

Sendo $x$ o produto final ($VBP$), e $(I - A)^{-1}$ a matriz B, ou matriz inversa de Leontief, que mostra os requisitos totais, diretos e indiretos, para a produção de cada setor. Podemos pensar nisso como uma análise de variação. 

$$\Delta x = (I - A)^{-1}\Delta y$$

Ou seja, uma variação na demanda final ($\Delta y$) causa um aumento na produção ($\Delta x$) de acordo com a tecnologia $(I - A)^{-1}$. Assim, assume-se que a economia é impulsionada por variações exógenas dado as relações endógenas.

- **Demanda Intermediária ($DI$):** É a soma das demandas internas dos setores da economia, consumida pelos setores. A produção total de setor, isto é, a soma dos valores de uma linha, equivale à Demanda Intermediária daquele setor.


- **Demanda Final ($DF$ ou $y_i$):** Demanda dos consumidores finais (famílias, governo, exportações e investimentos), exlui-se o que é consumido para confecção de novos produtos (a demanda intermediária).


- **Consumo Intermediário: ($CI$)** É o consumo total de insumos de cada setor. Em outras palavras, equivale à soma dos valores de uma determinada coluna.

- **Valor Adicionado da Produção ($VAB$):** É o valor que cada setor da economia acresce ao valor final de tudo que foi produzido em uma região. Uma das formas de calcular o Produto Interno Bruto ($PIB$) de uma economia é pela soma dos VABs setoriais e dos impostos. É a principal medida do tamanho total de uma economia.


- **Valor Bruto da Produção ($VBP$):** O Valor Bruto da Produção equivale à soma da produção total ($DI$) com a demanda final: $VBP = DI + DF$. Também pode ser calculado pela soma do Consumo Intermediário com o Valor Adicionado da Produção de cada setor: $VBP = CI + VAB$. Com isso, temos que a soma das demandas equivale à soma das produções: $DI + DF = CI + VAB$.


- **Encadeamentos para a frente (sensibilidade da dispersão):** relações de um setor B com outros setores como fornecedor.


- **Encadeamentos para trás (poder de dispersão):** relações de um setor C com outros setores como demandante.

## 1. Definição do Problema
---

A Matriz de Insumo-Produto é disponibilizada numa tabela em formato *.xlsx*, porém: 
- A tabela está no servidor de FTP 
- Os dados estão desalinhados, o que impossibilita o uso direto da tabela para análises de ligações. 

## 2. Obtenção dos dados
---

O primeiro passo é baixar os dados do IBGE. Para isso, acessamos o servidor FTP do IBGE, [neste link](https://ftp.ibge.gov.br/), e a partir dele podemos baixar o arquivo da Matriz de Insumo-Produto Nível 67. 

**Mas o que significa Nível 67?** Significa que a economia está dividida em 67 setores, desde agricultura até serviços domésticos.

Para baixarmos os dados diretos do site, utilizamos a biblioteca ``ftplib``:

In [1]:
from ftplib import FTP

No servidor FTP do IBGE há um problema de configuração do IP. Ele envia um IP interno para a rede externa da biblioteca ``ftplib``, que a classe FTP não consegue corrigir. Por isso, é necessária uma classe extra para corrigir tal problema.

In [2]:
class SmartFTP(FTP):
    
    def makepasv(self):
        invalidhost, port = super(SmartFTP, self).makepasv()
        return self.host, port

Com isso fora do caminho, podemos explorar o servidor FTP do IBGE.

In [3]:
ftp = SmartFTP('ftp.ibge.gov.br')
ftp.login()

'230 Login successful.'

Para facilitar a busca pela pasta certa, utilizaremos expressões regulares.

Expressões regulares (ou _regex_), são uma forma concisa e flexível de identificar cadeias de caracteres, sejam padrões, caracteres em particular ou palavras. Tal função está embutida no Python padrão com a biblioteca ``re``. 

In [4]:
import re

termo = re.compile('contas')
[pasta for pasta in ftp.nlst() if termo.search(pasta.lower())]

['Contas_Nacionais', 'Contas_Regionais']

In [5]:
ftp.cwd('Contas_Nacionais')
ftp.nlst()

['Contas_Nacionais_Trimestrais',
 'Financas_Publicas_e_Conta_Intermediaria_de_Governo',
 'Matriz_de_Insumo_Produto',
 'Sistema_de_Contas_Nacionais']

In [6]:
ftp.cwd('Matriz_de_Insumo_Produto')
ftp.nlst()

['1985',
 '1990',
 '1991',
 '1992',
 '1993',
 '1994',
 '1995',
 '1996',
 '2000',
 '2005',
 '2010',
 '2015']

In [7]:
ftp.cwd('2015')
ftp.nlst()

['Matriz_de_Insumo_Produto_2015_Nivel_12.ods',
 'Matriz_de_Insumo_Produto_2015_Nivel_12.xls',
 'Matriz_de_Insumo_Produto_2015_Nivel_20.ods',
 'Matriz_de_Insumo_Produto_2015_Nivel_20.xls',
 'Matriz_de_Insumo_Produto_2015_Nivel_67.ods',
 'Matriz_de_Insumo_Produto_2015_Nivel_67.xls',
 'indice_de_tabelas.txt']

Agora que já encontramos o arquivo desejado, precisamos baixá-lo para a pasta "Matrizes". Será utilizada a biblioteca ``pathlib`` para garantir que a pasta existe antes de tentar salvar arquivos nela.

In [8]:
from pathlib import Path

Path(f'{Path().resolve()}\Matrizes').mkdir(parents=True, exist_ok=True)

arquivo = 'Matriz_de_Insumo_Produto_2015_Nivel_67.xls'

with open('Matrizes/matriz_insumo_produto_67.xls', 'wb') as fobj:
    ftp.retrbinary('RETR %s' %arquivo, fobj.write)

Este arquivo possui múltiplas planilhas com dados. Precisamos descobrir qual nos interessa realmente. Para isso, há no servidor o índice de tabelas em uma arquivo txt. O baixaremos também. 

In [9]:
arquivo = 'indice_de_tabelas.txt'

with open('Matrizes/indice.txt', 'wb') as fobj:
    ftp.retrbinary('RETR %s' %arquivo, fobj.write)

Agora que finalizamos a utilização do FTP, devemos sair da conexão.

In [10]:
ftp.quit()

'221 Goodbye.'

## 3. Exploração dos dados
---
Agora, devemos analisar o indice de tabelas para ver onde estão os dados que queremos.

In [11]:
with open('Matrizes/indice.txt', 'r+') as file:
    texto = ''.join(file.readlines())
    
print(texto)

Sumário das tabelas_2015_nível 12
 
Tabela 01 - Recursos de bens e serviços - 2015
Tabela 02 - Usos de bens e serviços - 2015
Tabela 03 - Oferta e demanda da produção nacional a preço básico - 2015
Tabela 04 - Oferta e demanda de produtos importados a preço básico - 2015
Tabela 05 - Destino dos impostos sobre produtos nacionais - 2015
Tabela 06 - Destino dos impostos sobre produtos importados - 2015
Tabela 07 - Destino da margem de comércio sobre produtos nacionais - 2015
Tabela 08 - Destino da margem de comércio sobre produtos importados - 2015
Tabela 09 - Destino da margem de transporte sobre produtos nacionais - 2015
Tabela 10 - Destino da margem de transporte sobre produtos importados - 2015
Tabela 11 - Matriz dos coeficientes técnicos dos insumos nacionais - Matriz Bn - 2015
Tabela 12 - Matriz dos coeficientes técnicos dos insumos importados - Matriz Bm - 2015
Tabela 13 - Matriz de participação setorial na produção dos produtos nacionais - Matriz D - Market Share - 2015
Tabela 14 

Neste caso, queremos a tabela de oferta e demanda da produção nacional, a planilha 03. 

Agora temos os dados "crus". Estes dados mostram o valor monetário da produção de cada um dos setores, e em que parte da cadeia produtiva ele se encontra. Serão utilizados ``pandas`` e ``numpy`` para manipulação, exploração e análise dos dados.

In [12]:
import pandas as pd
import numpy as np

Antes de avançarmos, serão modificadas alguns parâmetros do ``paandas`` para ajudar na visualização das informações.

In [13]:
pd.options.display.max_columns = 500
pd.options.display.precision = 5

In [14]:
matriz = pd.read_excel("Matrizes/matriz_insumo_produto_67.xls",
                       sheet_name="03", 
                       skiprows=3)
matriz

,Unnamed: 0,Unnamed: 1,Valor da produção,"0191\nAgricultura, inclusive o apoio à agricultura e a pós-colheita","0192\nPecuária, inclusive o apoio à pecuária",0280\nProdução florestal; pesca e aquicultura,0580\nExtração de carvão mineral e de minerais não metálicos,"0680\nExtração de petróleo e gás, inclusive as atividades de apoio","0791\nExtração de minério de ferro, inclusive beneficiamentos e a aglomeração","0792\nExtração de minerais metálicos não ferrosos, inclusive beneficiamentos","1091\nAbate e produtos de carne, inclusive os produtos do laticínio e da pesca",1092\nFabricação e refino de açúcar,1093\nOutros produtos alimentares,1100\nFabricação de bebidas,1200\nFabricação de produtos do fumo,1300\nFabricação de produtos têxteis,1400\nConfecção de artefatos do vestuário e acessórios,1500\nFabricação de calçados e de artefatos de couro,1600\nFabricação de produtos da madeira,"1700\nFabricação de celulose, papel e produtos de papel",1800\nImpressão e reprodução de gravações,1991\nRefino de petróleo e coquerias,1992\nFabricação de biocombustíveis,"2091\nFabricação de químicos orgânicos e inorgânicos, resinas e elastômeros","2092\nFabricação de defensivos, desinfestantes, tintas e químicos diversos","2093\nFabricação de produtos de limpeza, cosméticos/perfumaria e higiene pessoal",2100\nFabricação de produtos farmoquímicos e farmacêuticos,2200\nFabricação de produtos de borracha e de material plástico,2300\nFabricação de produtos de minerais não metálicos,"2491\nProdução de ferro gusa/ferroligas, siderurgia e tubos de aço sem costura",2492\nMetalurgia de metais não ferosos e a fundição de metais,"2500\nFabricação de produtos de metal, exceto máquinas e equipamentos","2600\nFabricação de equipamentos de informática, produtos eletrônicos e ópticos",2700\nFabricação de máquinas e equipamentos elétricos,2800\nFabricação de máquinas e equipamentos mecânicos,"2991\nFabricação de automóveis, caminhões e ônibus, exceto peças",2992\nFabricação de peças e acessórios para veículos automotores,"3000\nFabricação de outros equipamentos de transporte, exceto veículos automotores",3180\nFabricação de móveis e de produtos de indústrias diversas,"3300\nManutenção, reparação e instalação de máquinas e equipamentos","3500\nEnergia elétrica, gás natural e outras utilidades","3680\nÁgua, esgoto e gestão de resíduos",4180\nConstrução,4580\nComércio por atacado e varejo,4900\nTransporte terrestre,5000\nTransporte aquaviário,5100\nTransporte aéreo,"5280\nArmazenamento, atividades auxiliares dos transportes e correio",5500\nAlojamento,5600\nAlimentação,5800\nEdição e edição integrada à impressão,"5980\nAtividades de televisão, rádio, cinema e gravação/edição de som e imagem",6100\nTelecomunicações,6280\nDesenvolvimento de sistemas e outros serviços de informação,"6480\nIntermediação financeira, seguros e previdência complementar",6800\nAtividades imobiliárias,"6980\nAtividades jurídicas, contábeis, consultoria e sedes de empresas","7180\nServiços de arquitetura, engenharia, testes/análises técnicas e P & D","7380\nOutras atividades profissionais, científicas e técnicas",7700\nAluguéis não imobiliários e gestão de ativos de propriedade intelectual,7880\nOutras atividades administrativas e serviços complementares,"8000\nAtividades de vigilância, segurança e investigação","8400\nAdministração pública, defesa e seguridade social",8591\nEducação pública,8592\nEducação privada,8691\nSaúde pública,8692\nSaúde privada,"9080\nAtividades artísticas, criativas e de espetáculos",9480\nOrganizações associativas e outros serviços pessoais,9700\nServiços domésticos,Total\ndo produto,Exportação\nde bens e\nserviços,Consumo\ndo governo,Consumo\ndas\n ISFLSF,Consumo \ndas famílias,Formação bruta\nde capital fixo,Variação\nde estoque,Demanda\nfinal,Demanda\ntotal
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [15]:
matriz.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132 entries, 0 to 131
Data columns (total 79 columns):
 #   Column                                                                             Non-Null Count  Dtype  
---  ------                                                                             --------------  -----  
 0   Unnamed: 0                                                                         129 non-null    object 
 1   Unnamed: 1                                                                         127 non-null    object 
 2   Valor da produção                                                                  128 non-null    float64
 3   0191
Agricultura, inclusive o apoio à agricultura e a pós-colheita                 128 non-null    float64
 4   0192
Pecuária, inclusive o apoio à pecuária                                        128 non-null    float64
 5   0280
Produção florestal; pesca e aquicultura                                       128 non-null    float64

In [16]:
print(f'Tamanho da matriz: {matriz.shape[0]} linhas x {matriz.shape[1]} colunas')
if matriz.shape[0] != matriz.shape[1]:
    print("Deveria ser quadrada!")

Tamanho da matriz: 132 linhas x 79 colunas
Deveria ser quadrada!


In [17]:
for coluna in matriz.columns:
    print(f'NaNs na coluna {coluna}: {matriz.loc[:, coluna].isnull().sum()} valores')

NaNs na coluna Unnamed: 0: 3 valores
NaNs na coluna Unnamed: 1: 5 valores
NaNs na coluna Valor da produção: 4 valores
NaNs na coluna 0191
Agricultura, inclusive o apoio à agricultura e a pós-colheita: 4 valores
NaNs na coluna 0192
Pecuária, inclusive o apoio à pecuária: 4 valores
NaNs na coluna 0280
Produção florestal; pesca e aquicultura: 4 valores
NaNs na coluna 0580
Extração de carvão mineral e de minerais não metálicos: 4 valores
NaNs na coluna 0680
Extração de petróleo e gás, inclusive as atividades de apoio: 4 valores
NaNs na coluna 0791
Extração de minério de ferro, inclusive beneficiamentos e a aglomeração: 4 valores
NaNs na coluna 0792
Extração de minerais metálicos não ferrosos, inclusive beneficiamentos: 4 valores
NaNs na coluna 1091
Abate e produtos de carne, inclusive os produtos do laticínio e da pesca: 4 valores
NaNs na coluna 1092
Fabricação e refino de açúcar: 4 valores
NaNs na coluna 1093
Outros produtos alimentares: 4 valores
NaNs na coluna 1100
Fabricação de bebidas

## 4. Tratamento dos dados

### Problemas a serem resolvidos: 

- Células com texto e valores ``NaN``;

- A maior parte dos nomes das colunas têm números e `\n`;

- Os setores estão dividos nas linhas mas não nas colunas (ou seja, não é uma matriz quadrada, o que impede a sua inversão)

Primeiro, apagamos as linhas desnecessárias no começo e no final da matriz

In [18]:
matriz.drop(index=[0, 128, 130, 131], inplace=True)
matriz

,Unnamed: 0,Unnamed: 1,Valor da produção,"0191\nAgricultura, inclusive o apoio à agricultura e a pós-colheita","0192\nPecuária, inclusive o apoio à pecuária",0280\nProdução florestal; pesca e aquicultura,0580\nExtração de carvão mineral e de minerais não metálicos,"0680\nExtração de petróleo e gás, inclusive as atividades de apoio","0791\nExtração de minério de ferro, inclusive beneficiamentos e a aglomeração","0792\nExtração de minerais metálicos não ferrosos, inclusive beneficiamentos","1091\nAbate e produtos de carne, inclusive os produtos do laticínio e da pesca",1092\nFabricação e refino de açúcar,1093\nOutros produtos alimentares,1100\nFabricação de bebidas,1200\nFabricação de produtos do fumo,1300\nFabricação de produtos têxteis,1400\nConfecção de artefatos do vestuário e acessórios,1500\nFabricação de calçados e de artefatos de couro,1600\nFabricação de produtos da madeira,"1700\nFabricação de celulose, papel e produtos de papel",1800\nImpressão e reprodução de gravações,1991\nRefino de petróleo e coquerias,1992\nFabricação de biocombustíveis,"2091\nFabricação de químicos orgânicos e inorgânicos, resinas e elastômeros","2092\nFabricação de defensivos, desinfestantes, tintas e químicos diversos","2093\nFabricação de produtos de limpeza, cosméticos/perfumaria e higiene pessoal",2100\nFabricação de produtos farmoquímicos e farmacêuticos,2200\nFabricação de produtos de borracha e de material plástico,2300\nFabricação de produtos de minerais não metálicos,"2491\nProdução de ferro gusa/ferroligas, siderurgia e tubos de aço sem costura",2492\nMetalurgia de metais não ferosos e a fundição de metais,"2500\nFabricação de produtos de metal, exceto máquinas e equipamentos","2600\nFabricação de equipamentos de informática, produtos eletrônicos e ópticos",2700\nFabricação de máquinas e equipamentos elétricos,2800\nFabricação de máquinas e equipamentos mecânicos,"2991\nFabricação de automóveis, caminhões e ônibus, exceto peças",2992\nFabricação de peças e acessórios para veículos automotores,"3000\nFabricação de outros equipamentos de transporte, exceto veículos automotores",3180\nFabricação de móveis e de produtos de indústrias diversas,"3300\nManutenção, reparação e instalação de máquinas e equipamentos","3500\nEnergia elétrica, gás natural e outras utilidades","3680\nÁgua, esgoto e gestão de resíduos",4180\nConstrução,4580\nComércio por atacado e varejo,4900\nTransporte terrestre,5000\nTransporte aquaviário,5100\nTransporte aéreo,"5280\nArmazenamento, atividades auxiliares dos transportes e correio",5500\nAlojamento,5600\nAlimentação,5800\nEdição e edição integrada à impressão,"5980\nAtividades de televisão, rádio, cinema e gravação/edição de som e imagem",6100\nTelecomunicações,6280\nDesenvolvimento de sistemas e outros serviços de informação,"6480\nIntermediação financeira, seguros e previdência complementar",6800\nAtividades imobiliárias,"6980\nAtividades jurídicas, contábeis, consultoria e sedes de empresas","7180\nServiços de arquitetura, engenharia, testes/análises técnicas e P & D","7380\nOutras atividades profissionais, científicas e técnicas",7700\nAluguéis não imobiliários e gestão de ativos de propriedade intelectual,7880\nOutras atividades administrativas e serviços complementares,"8000\nAtividades de vigilância, segurança e investigação","8400\nAdministração pública, defesa e seguridade social",8591\nEducação pública,8592\nEducação privada,8691\nSaúde pública,8692\nSaúde privada,"9080\nAtividades artísticas, criativas e de espetáculos",9480\nOrganizações associativas e outros serviços pessoais,9700\nServiços domésticos,Total\ndo produto,Exportação\nde bens e\nserviços,Consumo\ndo governo,Consumo\ndas\n ISFLSF,Consumo \ndas famílias,Formação bruta\nde capital fixo,Variação\nde estoque,Demanda\nfinal,Demanda\ntotal
1,01911,"Arroz, trigo e outros cereais",11036.0,166.0,164.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,8185.0,101.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

O setor "*0191 Agricultura, inclusive o apoio à agricultura e o pós-colheita*" está dividido em várias partes, mas todas possuem um código começando com 0191.

Isso permite a automatização da soma das divisões setoriais.

In [19]:
colunas = list(matriz.columns)

matriz_insumo_produto = pd.DataFrame(columns=matriz.columns)

for i in range(len(colunas)):
    indices = []
    
    for j in range(1, len(matriz.index)):
        
        subsetor = matriz.loc[j, 'Unnamed: 0']
        
        if subsetor[:4] in list(matriz.columns)[i]:
            # se o começo do código bate com o código desta coluna, marque-a para ser adicionada
            indices.append(matriz.index[j-1])
            
    if len(indices) != 0:
        # se há colunas para serem somadas, as some e as adicione na matriz_insumo_produto
        matriz_insumo_produto = matriz_insumo_produto.append(
            matriz.loc[indices[0]:indices[-1]].sum(),
            ignore_index=True
        )
        
matriz_insumo_produto

,Unnamed: 0,Unnamed: 1,Valor da produção,"0191\nAgricultura, inclusive o apoio à agricultura e a pós-colheita","0192\nPecuária, inclusive o apoio à pecuária",0280\nProdução florestal; pesca e aquicultura,0580\nExtração de carvão mineral e de minerais não metálicos,"0680\nExtração de petróleo e gás, inclusive as atividades de apoio","0791\nExtração de minério de ferro, inclusive beneficiamentos e a aglomeração","0792\nExtração de minerais metálicos não ferrosos, inclusive beneficiamentos","1091\nAbate e produtos de carne, inclusive os produtos do laticínio e da pesca",1092\nFabricação e refino de açúcar,1093\nOutros produtos alimentares,1100\nFabricação de bebidas,1200\nFabricação de produtos do fumo,1300\nFabricação de produtos têxteis,1400\nConfecção de artefatos do vestuário e acessórios,1500\nFabricação de calçados e de artefatos de couro,1600\nFabricação de produtos da madeira,"1700\nFabricação de celulose, papel e produtos de papel",1800\nImpressão e reprodução de gravações,1991\nRefino de petróleo e coquerias,1992\nFabricação de biocombustíveis,"2091\nFabricação de químicos orgânicos e inorgânicos, resinas e elastômeros","2092\nFabricação de defensivos, desinfestantes, tintas e químicos diversos","2093\nFabricação de produtos de limpeza, cosméticos/perfumaria e higiene pessoal",2100\nFabricação de produtos farmoquímicos e farmacêuticos,2200\nFabricação de produtos de borracha e de material plástico,2300\nFabricação de produtos de minerais não metálicos,"2491\nProdução de ferro gusa/ferroligas, siderurgia e tubos de aço sem costura",2492\nMetalurgia de metais não ferosos e a fundição de metais,"2500\nFabricação de produtos de metal, exceto máquinas e equipamentos","2600\nFabricação de equipamentos de informática, produtos eletrônicos e ópticos",2700\nFabricação de máquinas e equipamentos elétricos,2800\nFabricação de máquinas e equipamentos mecânicos,"2991\nFabricação de automóveis, caminhões e ônibus, exceto peças",2992\nFabricação de peças e acessórios para veículos automotores,"3000\nFabricação de outros equipamentos de transporte, exceto veículos automotores",3180\nFabricação de móveis e de produtos de indústrias diversas,"3300\nManutenção, reparação e instalação de máquinas e equipamentos","3500\nEnergia elétrica, gás natural e outras utilidades","3680\nÁgua, esgoto e gestão de resíduos",4180\nConstrução,4580\nComércio por atacado e varejo,4900\nTransporte terrestre,5000\nTransporte aquaviário,5100\nTransporte aéreo,"5280\nArmazenamento, atividades auxiliares dos transportes e correio",5500\nAlojamento,5600\nAlimentação,5800\nEdição e edição integrada à impressão,"5980\nAtividades de televisão, rádio, cinema e gravação/edição de som e imagem",6100\nTelecomunicações,6280\nDesenvolvimento de sistemas e outros serviços de informação,"6480\nIntermediação financeira, seguros e previdência complementar",6800\nAtividades imobiliárias,"6980\nAtividades jurídicas, contábeis, consultoria e sedes de empresas","7180\nServiços de arquitetura, engenharia, testes/análises técnicas e P & D","7380\nOutras atividades profissionais, científicas e técnicas",7700\nAluguéis não imobiliários e gestão de ativos de propriedade intelectual,7880\nOutras atividades administrativas e serviços complementares,"8000\nAtividades de vigilância, segurança e investigação","8400\nAdministração pública, defesa e seguridade social",8591\nEducação pública,8592\nEducação privada,8691\nSaúde pública,8692\nSaúde privada,"9080\nAtividades artísticas, criativas e de espetáculos",9480\nOrganizações associativas e outros serviços pessoais,9700\nServiços domésticos,Total\ndo produto,Exportação\nde bens e\nserviços,Consumo\ndo governo,Consumo\ndas\n ISFLSF,Consumo \ndas famílias,Formação bruta\nde capital fixo,Variação\nde estoque,Demanda\nfinal,Demanda\ntotal
0,019110191201913019140191501916019170191801919,"Arroz, trigo e outros cereaisMilho em grãoAlgo...",302321.0,6657.0,3635.0,78.0,0.0,0.0,0.0,0.0,910.0,25194.0,56751.0,1668.0,5870.0,2777.0,0.0,0.0,0.0,0.0,0.0,0

Agora corrigimos os nomes das colunas. Utilizaremos *regex* novamente.

In [20]:
colunas = list(matriz_insumo_produto.columns)
for i in range(len(colunas)):
    colunas[i] = re.sub('\d', "", colunas[i])  # Se há algum número no nome da coluna, o substitua por nada ("")
    colunas[i] = re.sub('\n', " ", colunas[i])  # Se há algum "\n" (pular para nova linha) o substituia por um espaço (" ")
    colunas[i] = colunas[i].strip()  # retira espaços em branco no começo e final da string
        
matriz_insumo_produto.columns = colunas

- VBP: Valor Bruto da Produção

- DF: Demanda Final

As variáveis estão escritas em caixa alta pois se tratam de valores constantes, seguindo a formatação recomendada na PEP 8.

In [21]:
VBP = matriz_insumo_produto["Demanda total"].copy()
DF = matriz_insumo_produto["Demanda final"].copy()

Retiramos agora as três primeiras colunas, pois não precisamos delas, e as duas últimas colunas, pois já guardamos estas informações em outras variáveis.

``inplace=True`` aplica o método `.drop()` na própria matriz

In [22]:
matriz_insumo_produto.drop(columns=(colunas[:3] + colunas[70:]),
                           axis=1,
                           inplace=True)

matriz_insumo_produto

,"Agricultura, inclusive o apoio à agricultura e a pós-colheita","Pecuária, inclusive o apoio à pecuária",Produção florestal; pesca e aquicultura,Extração de carvão mineral e de minerais não metálicos,"Extração de petróleo e gás, inclusive as atividades de apoio","Extração de minério de ferro, inclusive beneficiamentos e a aglomeração","Extração de minerais metálicos não ferrosos, inclusive beneficiamentos","Abate e produtos de carne, inclusive os produtos do laticínio e da pesca",Fabricação e refino de açúcar,Outros produtos alimentares,Fabricação de bebidas,Fabricação de produtos do fumo,Fabricação de produtos têxteis,Confecção de artefatos do vestuário e acessórios,Fabricação de calçados e de artefatos de couro,Fabricação de produtos da madeira,"Fabricação de celulose, papel e produtos de papel",Impressão e reprodução de gravações,Refino de petróleo e coquerias,Fabricação de biocombustíveis,"Fabricação de químicos orgânicos e inorgânicos, resinas e elastômeros","Fabricação de defensivos, desinfestantes, tintas e químicos diversos","Fabricação de produtos de limpeza, cosméticos/perfumaria e higiene pessoal",Fabricação de produtos farmoquímicos e farmacêuticos,Fabricação de produtos de borracha e de material plástico,Fabricação de produtos de minerais não metálicos,"Produção de ferro gusa/ferroligas, siderurgia e tubos de aço sem costura",Metalurgia de metais não ferosos e a fundição de metais,"Fabricação de produtos de metal, exceto máquinas e equipamentos","Fabricação de equipamentos de informática, produtos eletrônicos e ópticos",Fabricação de máquinas e equipamentos elétricos,Fabricação de máquinas e equipamentos mecânicos,"Fabricação de automóveis, caminhões e ônibus, exceto peças",Fabricação de peças e acessórios para veículos automotores,"Fabricação de outros equipamentos de transporte, exceto veículos automotores",Fabricação de móveis e de produtos de indústrias diversas,"Manutenção, reparação e instalação de máquinas e equipamentos","Energia elétrica, gás natural e outras utilidades","Água, esgoto e gestão de resíduos",Construção,Comércio por atacado e varejo,Transporte terrestre,Transporte aquaviário,Transporte aéreo,"Armazenamento, atividades auxiliares dos transportes e correio",Alojamento,Alimentação,Edição e edição integrada à impressão,"Atividades de televisão, rádio, cinema e gravação/edição de som e imagem",Telecomunicações,Desenvolvimento de sistemas e outros serviços de informação,"Intermediação financeira, seguros e previdência complementar",Atividades imobiliárias,"Atividades jurídicas, contábeis, consultoria e sedes de empresas","Serviços de arquitetura, engenharia, testes/análises técnicas e P & D","Outras atividades profissionais, científicas e técnicas",Aluguéis não imobiliários e gestão de ativos de propriedade intelectual,Outras atividades administrativas e serviços complementares,"Atividades de vigilância, segurança e investigação","Administração pública, defesa e seguridade social",Educação pública,Educação privada,Saúde pública,Saúde privada,"Atividades artísticas, criativas e de espetáculos",Organizações associativas e outros serviços pessoais,Serviços domésticos
0,6657.0,3635.0,78.0,0.0,0.0,0.0,0.0,910.0,25194.0,56751.0,1668.0,5870.0,2777.0,0.0,0.0,0.0,0.0,0.0,0.0,18138.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33.0,6.0,9641.0,0.0,0.0,0.0,0.0,285.0,3147.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,0.0,0.0,122.0,0.0,676.0,403.0,51.0,183.0,203.0,1.0,222.0,0.0
1,521.0,4475.0,45.0,0.0,0.0,0.0,0.0,87357.0,0.0,1589.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,279.0,1176.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,119.0,88.0,10.0,22.0,109.0,0.0,8.0,0.0
2,915.0,1095.0,1728.0,2.0,0.0,0.0,0.0,1406.0,0.0,212.0,1.0,4.0,39.0,10.0,14.0,2727.0,3681.0,0.0,0.0,1.0,162.0,0.0,0.0,0.0,887.0,85.0,764.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,687.0,240.0,0.0,0.0,0.0,0.0,38.0,333.0,0.0,0

Ao deixarmos todos os índices com o mesmo nome evitamos erros de processamento

In [23]:
matriz_insumo_produto.index = matriz_insumo_produto.columns
VBP.index = matriz_insumo_produto.columns
DF.index = matriz_insumo_produto.columns

Resolvido! Agora temos nossa matriz de insumo-produto pronta para ser manipulada!

Agora, importamos um pacote específico para análises de matriz de Insumo-Produto: o *pymrio*.

[*Pymrio: Multi-Regional Input-Output Analysis in Python - A python module for automating io calculations and generating reports*](https://github.com/konstantinstadler/pymrio)

Antes, porém, de utilizarmos o pacote, precisamos instalá-lo.

In [24]:
import sys
!conda install --yes --prefix {sys.prefix} -c conda-forge pymrio

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\ProgramData\Anaconda3

  added / updated specs:
    - pymrio


The following NEW packages will be INSTALLED:

  pymrio             conda-forge/noarch::pymrio-0.4.7-pyhd8ed1ab_0
  python_abi         conda-forge/win-64::python_abi-3.9-2_cp39

The following packages will be UPDATED:

  conda              pkgs/main::conda-4.13.0-py39haa95532_0 --> conda-forge::conda-4.13.0-py39hcbf5309_1


Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done


A instalação é feita da forma demonstrada acima para garantir que será instalado apenas no *kernel* atual.

In [25]:
import pymrio

In [26]:
matriz_A = pymrio.calc_A(matriz_insumo_produto, VBP)
matriz_A

,"Agricultura, inclusive o apoio à agricultura e a pós-colheita","Pecuária, inclusive o apoio à pecuária",Produção florestal; pesca e aquicultura,Extração de carvão mineral e de minerais não metálicos,"Extração de petróleo e gás, inclusive as atividades de apoio","Extração de minério de ferro, inclusive beneficiamentos e a aglomeração","Extração de minerais metálicos não ferrosos, inclusive beneficiamentos","Abate e produtos de carne, inclusive os produtos do laticínio e da pesca",Fabricação e refino de açúcar,Outros produtos alimentares,Fabricação de bebidas,Fabricação de produtos do fumo,Fabricação de produtos têxteis,Confecção de artefatos do vestuário e acessórios,Fabricação de calçados e de artefatos de couro,Fabricação de produtos da madeira,"Fabricação de celulose, papel e produtos de papel",Impressão e reprodução de gravações,Refino de petróleo e coquerias,Fabricação de biocombustíveis,"Fabricação de químicos orgânicos e inorgânicos, resinas e elastômeros","Fabricação de defensivos, desinfestantes, tintas e químicos diversos","Fabricação de produtos de limpeza, cosméticos/perfumaria e higiene pessoal",Fabricação de produtos farmoquímicos e farmacêuticos,Fabricação de produtos de borracha e de material plástico,Fabricação de produtos de minerais não metálicos,"Produção de ferro gusa/ferroligas, siderurgia e tubos de aço sem costura",Metalurgia de metais não ferosos e a fundição de metais,"Fabricação de produtos de metal, exceto máquinas e equipamentos","Fabricação de equipamentos de informática, produtos eletrônicos e ópticos",Fabricação de máquinas e equipamentos elétricos,Fabricação de máquinas e equipamentos mecânicos,"Fabricação de automóveis, caminhões e ônibus, exceto peças",Fabricação de peças e acessórios para veículos automotores,"Fabricação de outros equipamentos de transporte, exceto veículos automotores",Fabricação de móveis e de produtos de indústrias diversas,"Manutenção, reparação e instalação de máquinas e equipamentos","Energia elétrica, gás natural e outras utilidades","Água, esgoto e gestão de resíduos",Construção,Comércio por atacado e varejo,Transporte terrestre,Transporte aquaviário,Transporte aéreo,"Armazenamento, atividades auxiliares dos transportes e correio",Alojamento,Alimentação,Edição e edição integrada à impressão,"Atividades de televisão, rádio, cinema e gravação/edição de som e imagem",Telecomunicações,Desenvolvimento de sistemas e outros serviços de informação,"Intermediação financeira, seguros e previdência complementar",Atividades imobiliárias,"Atividades jurídicas, contábeis, consultoria e sedes de empresas","Serviços de arquitetura, engenharia, testes/análises técnicas e P & D","Outras atividades profissionais, científicas e técnicas",Aluguéis não imobiliários e gestão de ativos de propriedade intelectual,Outras atividades administrativas e serviços complementares,"Atividades de vigilância, segurança e investigação","Administração pública, defesa e seguridade social",Educação pública,Educação privada,Saúde pública,Saúde privada,"Atividades artísticas, criativas e de espetáculos",Organizações associativas e outros serviços pessoais,Serviços domésticos
"Agricultura, inclusive o apoio à agricultura e a pós-colheita",0.02202,0.02820,0.00229,0.00000,0.00000,0.00000,0.00000,0.00350,0.70485,0.20577,0.02133,0.38882,0.05776,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.30863,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00051,9.30834e-06,9.29697e-03,0.00000,0.00000,0.00000,0.00000,0.01170,0.01397,0.0000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00002,3.40779e-05,0.00000,0.00000,0.00059,0.00000,0.00101,0.00140,0.00045,0.00104,9.02403e-04,0.00003,0.00146,0.0
"Pecuária, inclusive o apoio à pecuária",0.00172,0.03471,0.00132,0.00000,0.00000,0.00000,0.00000,0.33551,0.00000,0.00576,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00

In [27]:
matriz_L = pymrio.calc_L(matriz_A)
matriz_L

,"Agricultura, inclusive o apoio à agricultura e a pós-colheita","Pecuária, inclusive o apoio à pecuária",Produção florestal; pesca e aquicultura,Extração de carvão mineral e de minerais não metálicos,"Extração de petróleo e gás, inclusive as atividades de apoio","Extração de minério de ferro, inclusive beneficiamentos e a aglomeração","Extração de minerais metálicos não ferrosos, inclusive beneficiamentos","Abate e produtos de carne, inclusive os produtos do laticínio e da pesca",Fabricação e refino de açúcar,Outros produtos alimentares,Fabricação de bebidas,Fabricação de produtos do fumo,Fabricação de produtos têxteis,Confecção de artefatos do vestuário e acessórios,Fabricação de calçados e de artefatos de couro,Fabricação de produtos da madeira,"Fabricação de celulose, papel e produtos de papel",Impressão e reprodução de gravações,Refino de petróleo e coquerias,Fabricação de biocombustíveis,"Fabricação de químicos orgânicos e inorgânicos, resinas e elastômeros","Fabricação de defensivos, desinfestantes, tintas e químicos diversos","Fabricação de produtos de limpeza, cosméticos/perfumaria e higiene pessoal",Fabricação de produtos farmoquímicos e farmacêuticos,Fabricação de produtos de borracha e de material plástico,Fabricação de produtos de minerais não metálicos,"Produção de ferro gusa/ferroligas, siderurgia e tubos de aço sem costura",Metalurgia de metais não ferosos e a fundição de metais,"Fabricação de produtos de metal, exceto máquinas e equipamentos","Fabricação de equipamentos de informática, produtos eletrônicos e ópticos",Fabricação de máquinas e equipamentos elétricos,Fabricação de máquinas e equipamentos mecânicos,"Fabricação de automóveis, caminhões e ônibus, exceto peças",Fabricação de peças e acessórios para veículos automotores,"Fabricação de outros equipamentos de transporte, exceto veículos automotores",Fabricação de móveis e de produtos de indústrias diversas,"Manutenção, reparação e instalação de máquinas e equipamentos","Energia elétrica, gás natural e outras utilidades","Água, esgoto e gestão de resíduos",Construção,Comércio por atacado e varejo,Transporte terrestre,Transporte aquaviário,Transporte aéreo,"Armazenamento, atividades auxiliares dos transportes e correio",Alojamento,Alimentação,Edição e edição integrada à impressão,"Atividades de televisão, rádio, cinema e gravação/edição de som e imagem",Telecomunicações,Desenvolvimento de sistemas e outros serviços de informação,"Intermediação financeira, seguros e previdência complementar",Atividades imobiliárias,"Atividades jurídicas, contábeis, consultoria e sedes de empresas","Serviços de arquitetura, engenharia, testes/análises técnicas e P & D","Outras atividades profissionais, científicas e técnicas",Aluguéis não imobiliários e gestão de ativos de propriedade intelectual,Outras atividades administrativas e serviços complementares,"Atividades de vigilância, segurança e investigação","Administração pública, defesa e seguridade social",Educação pública,Educação privada,Saúde pública,Saúde privada,"Atividades artísticas, criativas e de espetáculos",Organizações associativas e outros serviços pessoais,Serviços domésticos
"Agricultura, inclusive o apoio à agricultura e a pós-colheita",1.02752,0.05609,0.01015,0.00484,0.00353,0.00353,0.00504,0.05157,0.75318,0.25090,0.04627,0.42127,0.07619,0.01905,0.01339,0.00415,0.00597,0.00283,0.03359,0.33870,0.00541,0.01132,0.02538,0.00783,0.00423,0.00451,0.00403,0.00436,0.00297,0.00228,0.00323,0.00274,0.00366,0.00430,0.00260,0.00413,0.00210,0.00247,0.00214,0.00260,0.01232,0.00943,0.00247,0.00629,0.00170,0.02005,0.03796,0.00211,0.00168,0.00127,0.00066,0.00087,0.00016,0.00099,0.00097,0.00139,0.00164,0.00178,0.00131,0.00323,0.00420,0.00167,0.00542,0.00419,0.00129,0.00710,0.0
"Pecuária, inclusive o apoio à pecuária",0.00200,1.04104,0.00176,0.00016,0.00020,0.00016,0.00021,0.38140,0.00159,0.01007,0.00055,0.00092,0.00027,0.00014,0.03317,0.00034,0.00035,0.00016,0.00065,0.00581,0.00019,0.00039,0.01561,0.00038,0.00018,0.00018,0.0

Chegamos enfim à matriz de coeficientes!

Agora podemos exportar estas matrizes para arquivos csv, para evitar calcularmos novamente estes valores

In [28]:
matriz_insumo_produto.to_csv("Matrizes\matriz_insumo_produto.csv", header=True)
matriz_A.to_csv("Matrizes\matriz_A.csv", header=True)
matriz_L.to_excel("Matrizes\matriz_coeficientes.xlsx")
VBP.to_csv("Matrizes\VBP.csv", header=True)
DF.to_csv("Matrizes\demanda_final.csv", header=True)

## 5. Análise dos dados

---

Com a matriz de coeficientes pronta, podemos realizar as análises e descobrir o tipo dos setores

Os tipos dependem do coeficiente das ligações para frente e para trás. Calculamos isso da seguinte forma:

$$L_f = \frac{\frac{1}{n}\sum_{j}{b_{ij}}}{\frac{1}{n^2}\sum_i\sum_j b_{ij}}$$


$$L_t = \frac{\frac{1}{n}\sum_{i}{b_{ij}}}{\frac{1}{n^2}\sum_i\sum_j b_{ij}}$$

Ou seja: o coeficiente de ligação para frente da linha $i$ é igual à média dos coeficientes desta linha dividida pela média de todos os coeficientes da matriz;

E o coeficiente de ligações para trás da coluna $j$ é igual à média dos coeficientes desta coluna dividida pela média total dos coeficientes.

Eu realizo a transformação para uma matriz com `.to_numpy()` antes de tirar a média de todos os valores pois é uma operação muito mais eficiente. O método `.mean()` em pandas apenas realiza a média de todas as linhas, então precisaríamos realizar a média duas vezes.

In [17]:
media = matriz_L.to_numpy(copy=True).mean()

l_f = matriz_L.mean(axis=0) / media # axis=0 para média das colunas
l_t = matriz_L.mean(axis=1) / media # axis=1 para média das colunas

In [18]:
df_coeficientes = pd.DataFrame({
    "Setor": list(matriz_L.columns),
    "Ligações para frente": l_f.to_numpy(),
    "Ligações para trás": l_t.to_numpy()
})
df_coeficientes

,Setor,Ligações para frente,Ligações para trás
0,"Agricultura, inclusive o apoio à agricultura e...",0.965247,1.862446
1,"Pecuária, inclusive o apoio à pecuária",1.018823,0.870680
2,Produção florestal; pesca e aquicultura,0.742157,0.762387
3,Extração de carvão mineral e de minerais não m...,0.995418,0.688005
4,"Extração de petróleo e gás, inclusive as ativi...",0.963418,1.406660
...,...,...,...
62,Saúde pública,0.814404,0.553941
63,Saúde privada,0.840359,0.611399
64,"Atividades artísticas, criativas e de espetáculos",0.873071,0.640074
65,Organizações associativas e outros serviços pe...,0.902033,0.724536


Podemos dividir os setores em quatro tipos:

- **Tipo I:** ($L_t < 1; L_f < 1$) Geralmente independente, sem ligações ou ligações fracas;

- **Tipo II:** ($L_t < 1; L_f > 1$) Depende da demanda interisdustrial, mais relações para frente;

- **Tipo III:** ($L_t > 1; L_f > 1$) Geralmente dependente, é um ***setor chave***, contribui acima da média para o crescimento da economia;

- **Tipo IV:** ($L_t > 1; L_f < 1$) Depende da oferta interindustrial, mais relações para trás;

In [22]:
tipos = []
chaves = []

for i in range(df_coeficientes.shape[0]):
    chave = ''
    if df_coeficientes.iloc[i, 1] < 1 and df_coeficientes.iloc[i, 2] < 1:
        tipo = 'I'
    elif df_coeficientes.iloc[i, 1] < 1 and df_coeficientes.iloc[i, 2] > 1:
        tipo = 'II'
    elif df_coeficientes.iloc[i, 1] > 1 and df_coeficientes.iloc[i, 2] > 1:
        tipo = 'III'
        chave = '*'
    else:
        tipo = 'IV'
    tipos.append(tipo)
    chaves.append(chave)

df_coeficientes['Tipo'] = tipos
df_coeficientes['Setor Chave'] = chaves
df_coeficientes[df_coeficientes['Setor Chave'] == '*']

,Setor,Ligações para frente,Ligações para trás,Tipo,Setor Chave
16,"Fabricação de celulose, papel e produtos de papel",1.132302,1.008885,III,*
18,Refino de petróleo e coquerias,1.350624,2.823324,III,*
20,Fabricação de químicos orgânicos e inorgânicos...,1.115801,1.909688,III,*
21,"Fabricação de defensivos, desinfestantes, tint...",1.121799,1.134049,III,*
24,Fabricação de produtos de borracha e de materi...,1.143552,1.202437,III,*
26,"Produção de ferro gusa/ferroligas, siderurgia ...",1.222849,1.043301,III,*
28,"Fabricação de produtos de metal, exceto máquin...",1.077724,1.053088,III,*
37,"Energia elétrica, gás natural e outras utilidades",1.162158,2.027143,III,*
41,Transporte terrestre,1.121909,2.251273,III,*


É notável um padrão entre os possíveis setores chave: *a maioria deles é do setor industrial*. Um ponto muito comentado por economistas desenvolvimentistas é a importância da retomada do crescimento deste setor no Brasil para alavancar a economia.

Uma relação interessante que podemos visualizar é a seguinte:

$$L \cdot DF = VBP$$

Isto é, a multiplicação da matriz de coeficientes pelo vetor da Demanda Final resulta no VBP! Com isso, podemos prever qual seria a variação do VBP ao mudarmos cada valor no DF.

Vejamos, qual seria o impactor no VBP se a Demanda Final apenas do primeiro setor subisse em 10% e dos outros permanecesse constante, considerando a matriz de coeficientes que nós temos?

In [20]:
multiplicador = [1.1]
multiplicador.extend([1] * 66)

DF_2 = DF.copy() * multiplicador
DF_2.index = matriz_L.columns
VBP_2 = matriz_L.dot(DF_2)

(VBP_2/VBP)

Agricultura, inclusive o apoio à agricultura e a pós-colheita    1.056305
Pecuária, inclusive o apoio à pecuária                           1.000258
Produção florestal; pesca e aquicultura                          1.001947
Extração de carvão mineral e de minerais não metálicos           1.002366
Extração de petróleo e gás, inclusive as atividades de apoio     1.002415
                                                                   ...   
Saúde pública                                                    1.000000
Saúde privada                                                    1.000000
Atividades artísticas, criativas e de espetáculos                1.000071
Organizações associativas e outros serviços pessoais             1.000184
Serviços domésticos                                              1.000000
Length: 67, dtype: float64

Podemos ver, como esperado, que a variação no produto total é diferente em cada setor e não batem com a variação na demanda. 